### Silver Layer: Process circuits data

Read from bronze/raw layer (parquet) and transform for silver/processed layer.

In [ ]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [ ]:
from formula1.formula1_constants import raw_folder_path, processed_folder_path

In [ ]:
from formula1.formula1_utils import add_ingestion_date

##### Step 1 - Read parquet from bronze/raw layer

In [ ]:
circuits_df = spark.read.parquet(f"{raw_folder_path}/circuits")

##### Step 2 - Select only the required columns

In [ ]:
from pyspark.sql.functions import col

In [ ]:
circuits_selected_df = circuits_df.select(
    col("circuitId"), col("circuitRef"), col("name"), 
    col("location"), col("country"), col("lat"), col("lng"), col("alt"),
    col("ingestion_date"), col("data_source")
)

##### Step 3 - Rename the columns as required

In [ ]:
circuits_renamed_df = circuits_selected_df \
    .withColumnRenamed("circuitId", "circuit_id") \
    .withColumnRenamed("circuitRef", "circuit_ref") \
    .withColumnRenamed("lat", "latitude") \
    .withColumnRenamed("lng", "longitude") \
    .withColumnRenamed("alt", "altitude")

##### Step 4 - Update ingestion date for silver layer

In [ ]:
circuits_final_df = add_ingestion_date(circuits_renamed_df)

##### Step 5 - Write data to silver/processed layer as parquet

In [ ]:
circuits_final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/circuits")